In [1]:
import socket
import sys
import bisect
from copy import deepcopy
import time
import datetime
import threading

In [2]:
class sorted_set(object):
    def __init__(self):
        self.scores = []
        self.values = []
        self.size = 0
        self.min_score = 0
    def index(self,a, x):
#         print(a)
#         print(x)
        i = bisect.bisect_left(a, x)
        if i != len(a) and a[i][0] == x[0]:
            return i
        return None

    def find_index_of_value(self,value):
        return self.index(self.values , (value, self.min_score-1))
    
    def find_index_of_score(self,score, value=None):
        return self.index(self.scores, (score,value))
    
    def zadd(self,list_of_score_value):
        count = 0
        for score,value in list_of_score_value:
            if self.size == 0:
                self.scores.append((score,value))
                self.values.append((value,score))
                self.size += 1
                self.min_score = score
                count += 1
            else:
                self.min_score = min(self.min_score ,score)
                index_of_value = self.find_index_of_value(value)
                if index_of_value == None:
                    bisect.insort_left(self.values, (value, score))
                    bisect.insort_left(self.scores, (score, value))
                    self.size += 1
                    count += 1
                else:
                    old_score = self.values[index_of_value][1]
                    old_score_index = self.find_index_of_score(old_score,value)
                    self.scores.pop(old_score_index)
#                     print(index_of_value,"index")
                    self.values[index_of_value] = (value,score)
                    bisect.insort_left(self.scores, (score, value))
        return count
    def zrank(self,value):
        index_of_value = self.find_index_of_value(value)
        if index_of_value is None:
            return -1
        else:
            return self.find_index_of_score(self.values[index_of_value][1],value)
    
    def zrange(self, left , right , reverse = False , with_scores = False):
        if right < 0:
            right = self.size + right + 1
        scores_copy = deepcopy(self.scores)
        if reverse:
            scores_copy = list(reversed(scores_copy))
        if with_scores:
            return [(value,score) for (score,value) in scores_copy[left:right]]
        else:
            return [(value) for (score,value) in scores_copy[left:right]]

In [3]:
class String:
    def get(self):
        return self.val

    def set(self, val):
        self.val = val

In [4]:
class hash_table:
    def __init__(self):
        self.hash_table = {}
        self.expired_time_for_keys = {}
        threading.Thread(target=self.thread_run_fun).start()
        
    def set(self,key,value,expired=None):
        strobj = String()
        strobj.set(value)
        self.hash_table[key] = strobj
        if expired is not None:
            self.expired_time_for_keys[key] = datetime.datetime.now()+datetime.timedelta(0,expired)
        else:
            if key in self.expired_time_for_keys:
                self.expired_time_for_keys.pop(key)
                
    def get(self,key):
        if key in self.hash_table:
#             print("Inside")
            return self.hash_table[key].get()
        else:
            return None
        
    def expire(self, key, expired):
        if key in self.hash_table and key not in self.expired_time_for_keys:
            self.expired_time_for_keys[key] = datetime.datetime.now()+datetime.timedelta(0,expired)
            return "1"
        else:
            return "0"
                
    
    def zadd(self,key,list_of_score_value):
        if key in self.hash_table:
            count = self.hash_table[key].zadd(list_of_score_value)
        else:
            ssobj = sorted_set()
            count = ssobj.zadd(list_of_score_value)
            self.hash_table[key] = ssobj
            
        return count
            
    def zrank(self,key,value):
        if key in self.hash_table:
            return self.hash_table[key].zrank(value)
        else:
            return None
        
    
    def zrange(self,key,left,right,reverse=False,with_score=False):
        if key in self.hash_table:
            return self.hash_table[key].zrange(left,right,reverse,with_score)
        else:
            return None
        
    def thread_run_fun(self):
#         print("here")
        while True:
            new_dic = deepcopy(self.expired_time_for_keys)
            for key in new_dic:
                if self.expired_time_for_keys[key] >= datetime.datetime.now():
                    self.hash_table.pop(key)
                    self.expired_time_for_keys.pop(key)
            time.sleep(10)

In [ ]:
obj = hash_table()

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 8080
s.bind(('0.0.0.0', port))
s.listen(5)

c, addr = s.accept()
# print('Got connection from', addr)

while True:
#     print("Inside")
    try:
        command = str(c.recv(1024), "utf-8")
#         print(command)

        arr = command.split()

        if arr[0] == 'SET':
            if len(arr) > 3 and arr[3] == "EXP":
                obj.set(arr[1], arr[2], int(arr[4]))
                c.sendall("OK".encode('utf-8'))
            else:
                obj.set(arr[1], arr[2])
                c.sendall("OK".encode('utf-8'))
        if arr[0] == "GET":
            val = obj.get(arr[1])
            if val is None:
                val = "(nil)"
            c.sendall(val.encode('utf-8'))
        if arr[0] == "EXPIRE":
            val = obj.expire(arr[1], int(arr[2]))
            c.sendall(val.encode('utf-8'))
        if arr[0] == "ZADD":
            lst = []
            for i in range(2, len(arr), 2):
                lst.append((int(arr[i]), arr[i + 1]))
#             print(lst)
            val = str(obj.zadd(arr[1], lst))
            c.sendall(val.encode('utf-8'))
        if arr[0] == "ZRANK":
            val = str(obj.zrank(arr[1],arr[2]))
            if val == "-1":
                val = "(nil)"
            c.sendall(val.encode('utf-8'))
        if arr[0] == "ZRANGE":
            temp = ''
            if "WITHSCORES" in command:
                val = obj.zrange(arr[1],int(arr[2]),int(arr[3]), False, True)
                for atuple in val:
                    temp += atuple[0] + "-" + str(atuple[1])
                    temp += ' '
            else:
                val = obj.zrange(arr[1],int(arr[2]),int(arr[3]))
                for key in val:
                    temp += key
                    temp += ' '
            c.sendall(temp.encode('utf-8'))
    except socket.error:
        print("Error Occured.")
        break
        
c.close()

here
Got connection from ('127.0.0.1', 52588)
Inside
GET vishal foo
Inside
SET vishal foo
Inside
GET vishal foo
Inside
Inside
SET key1 bar EXP 10
Inside
EXPIRE key2 30
Inside
EXPIRE vishal 30
Inside
GET vishal
Inside
ZADD myzset 1 "one" 2 "two" 3 "three"
[(1, '"one"'), (2, '"two"'), (3, '"three"')]
[('"one"', 1)]
('"two"', 0)
[('"one"', 1), ('"two"', 2)]
('"three"', 0)
Inside
ZRANK myzset "two"
[('"one"', 1), ('"three"', 3), ('"two"', 2)]
('"two"', 0)
[(1, '"one"'), (2, '"two"'), (3, '"three"')]
(2, '"two"')
Inside
ZADD myzset 4 "four" 5 "five"
[(4, '"four"'), (5, '"five"')]
[('"one"', 1), ('"three"', 3), ('"two"', 2)]
('"four"', 0)
[('"four"', 4), ('"one"', 1), ('"three"', 3), ('"two"', 2)]
('"five"', 0)
Inside
ZRANGE myzset 2 4
Inside
ZRANGE myzset 2 5 WITHSCORES
Inside
ZRANGE myzset 0 -1
Inside
